In [ ]:
#KO H06
#satunnaismetsä

In [16]:
import  pandas as pd

#luetaan data
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00597/garments_worker_productivity.csv'
df = pd.read_csv(url)

#tipautetaan oletettavasti merkityksettömät taustamuuttujat
df.drop(columns=['date','day','quarter','department','team'], inplace=True)

#tehdään luokkamuuttujaa varten pieni funktio
def productivity_to_class(df):
    if df['actual_productivity'] >= 0.5:
        return 1
    else:
        return 0

#ja luodaan luokkamuuttuja 
df['class'] = df.apply(lambda x: productivity_to_class(x), axis=1)

#täytetään NaNit sarakkeen keskiarvolla
fill_value = df['wip'].mean()
df['wip'].fillna(fill_value, inplace=True)

#eriytetään luokka ja tuottavuus
#indeksit tulevan tarpeen mukaisessa järjestyksessä
y1 = df['actual_productivity']
y2 = df['class']
X = df.drop(columns=['actual_productivity','class'])

print(f'Columns: {df.columns}')
print(f'First row by position: {df.iloc[0]}')

Columns: Index(['targeted_productivity', 'smv', 'wip', 'over_time', 'incentive',
       'idle_time', 'idle_men', 'no_of_style_change', 'no_of_workers',
       'actual_productivity', 'class'],
      dtype='object')
First row by position: targeted_productivity       0.800000
smv                        26.160000
wip                      1108.000000
over_time                7080.000000
incentive                  98.000000
idle_time                   0.000000
idle_men                    0.000000
no_of_style_change          0.000000
no_of_workers              59.000000
actual_productivity         0.940725
class                       1.000000
Name: 0, dtype: float64


# oma kommentti
Olettaisin, ettei sarakkeilla `'day'`,`'date'`,`'quarter'`,`'department'` ja `'team'` ole analyysissä hirveän suurta painoarvoa ja droppasin ne. (Viikonpäivän tosin voisi ajatella vaikuttavan, ainakin jos viikonloppu lähestyy. Datan taustatiedoista jää kuitenkin epäselväksi milloin viikonloppu on, ja onko tutkituilla edes vapaata silloin.)

Work in progress (`wip`) -sarakkeessa oli puuttuvia arvoja. Koska niiden määrä on tyypillisesti melko suuri, korvasin puuttuvat arvot sarakkeen keskiarvolla, jolloin ne eivät (likimain normaalijakauma olettaen) heiluta sarakkeen painoa.

Algoritmia varten tehtiin juoksevasta `actual_productivity` -arvosta luokkamuuttuja `class`, jonka arvo on yksi, jos työläinen oli keskimääräistä tuottavampi ja nolla muutoin. Taaskaan ei normalisoida ennen datan jakamista.

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import explained_variance_score, max_error

#jaetaan data. tehdään tässä kerralla molemmat y:t
X_train, X_test, y1_train, y1_test, y2_train, y2_test = train_test_split(X, y1, y2, test_size=0.33, random_state=23)

#ja nyt normalisoidaan
sts = StandardScaler()
X_train = pd.DataFrame(sts.fit_transform(X_train))
X_test = pd.DataFrame(sts.transform(X_test))

#koulutetaan satunnaismetsä
rfr = RandomForestRegressor(random_state=753) #n_estimators=100, max_depth=???
rfr.fit(X_train, y1_train)

#ennuste
y1_pred = rfr.predict(X_test)

#selittyvä varianssi sekä maksimivirhe
exp_var = explained_variance_score(y1_test, y1_pred)
max_err = max_error(y1_test, y1_pred)

print(f'Test length: {len(X_test)}')
print(f'Train length: {len(X_train)}')
print(f'Train columns: {X_train.columns}')
print(f'Explained variance: {exp_var}')
print(f'Maximum error: {max_err}')


Test length: 396
Train length: 801
Train columns: RangeIndex(start=0, stop=9, step=1)
Explained variance: 0.3686123761563034
Maximum error: 0.587630922640574


# oma kommentti
Tässä tehtiin kerralla kaikki tehtäväpaketissa tarvittavat `y`-aineistot ja sen jälkeen normalisoitiin `X`-aineistot. Vaikka päätöspuu ei olekaan herkkä datan arvojen erisuuruuksille, tässä tehtävässä regressiossa normalisointi voi olla fiksua, koska suureiden skaalaaminen samalle asteikolle joka tapauksessa parantaa mallin yleistämiskykyä. Skaalausta ei tehty  `actual_productivity`:lle. Vaikka se onkin jatkuva suure, on vaikea keksiä, mitä sillä saavutettaisiin. 

Päätöspuiden lukumäärää tai maksimisyvyyttä ei ole annettu, joten mennään oletusarvoilla, jolloin puita on 100 ja niitä kasvatetaan kunnes ne saavuttavat luonnollisen maksimin. Lopputulosta tarkastellaan `metrics`-moduulin funktioilla. Päätösuu on konseptina tuttu, mutta regressioanalyysin tekeminen niiden avulla on ajatuksena hieman vieras.

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

#koulutetaan luokittelija
rfc = RandomForestClassifier(random_state=753)
rfc.fit(X_train, y2_train)

#ennustetaan
y2_pred = rfc.predict(X_test)

#tarkastellaan onnistumista
cr = classification_report(y2_test, y2_pred)
cm = confusion_matrix(y2_test, y2_pred)

print(f'Test length: {len(X_test)}')
print(f'Train length: {len(X_train)}')
print(cr)
print(cm)

#katsotaan vielä mikä oikeastaan vaikutti luokitteluun
importances = rfc.feature_importances_
for i in range(len(importances)):
    print("feature: %12s - relative importance: %4.1f %%" % (X.columns[i], importances[i] * 100))


Test length: 396
Train length: 801
              precision    recall  f1-score   support

           0       0.44      0.43      0.43        35
           1       0.94      0.95      0.95       361

    accuracy                           0.90       396
   macro avg       0.69      0.69      0.69       396
weighted avg       0.90      0.90      0.90       396

[[ 15  20]
 [ 19 342]]
feature: targeted_productivity - relative importance: 22.3 %
feature:          smv - relative importance: 18.9 %
feature:          wip - relative importance: 12.1 %
feature:    over_time - relative importance: 17.7 %
feature:    incentive - relative importance: 11.0 %
feature:    idle_time - relative importance:  1.5 %
feature:     idle_men - relative importance:  2.2 %
feature: no_of_style_change - relative importance:  1.5 %
feature: no_of_workers - relative importance: 12.9 %


# oma kommentti
Viimeisessä tehtävässä satunnaismetsä-luokittelijan piti ennustaa `class`-luokkamuuttuja ja sehän onnistui kohtalaisen hyvin. Luentomateriaaleista löytyi kiva suhteellisen merkittävyyden työkalu, joka kertoo tehokkuuteen vaikuttavan erityisesti asetettu tuottavuustavoite (`targeted_productivity`) sekä työhön varattu aika ja sen ylitys (`smv` ja `over_time`). Taloudellinen kannuste (`incentive`) jää omasta mielestäni yllättävän matalalle painolle. Tekemistä varmaankin siis riittää. Tulokset vaikuttavat kokonaisuutena ihan mielekkäiltä.